# GPT1 - Improving Language Understanding by Generative Pre-Training

## Resumen

 * Entrenan el modelo en un gran corpus de texto sin supervisión. Modelado del lenguaje. Aprender un modelo de lenguaje de alta capacidad en un gran corpus de texto
 * Luego hacen fine-tuning en tareas de NLP supervisadas. Es decir con datasets etiquetados. Ajuste fino en una tarea objetivo con supervisión. Ajuste fino, en la que adaptamos el modelo a una tarea discriminativa con datos etiquetados. CUando evaluan al modelo en la tarea supervisada, no solo le evaluan por esa tarea, sino por lo bien que predice el siguiente token, esto ayuda a mejorar la generalización del modelo supervisado y hace que el modelo converja más rápido.
 * Utilizan la arquitectura transformer, que mejora al uso de RNN en que lo aprendido en el primer entrenamiento es más fácil de transferir a tareas supervisadas.
 * Evaluan el modelo en cuatro tipos de tareas de comprensión del lenguaje:
    * Inferencia del lenguaje natural
    * Respuesta a preguntas
    * Similitud semántica
    * Clasificación de textos. 
 * El modelo, agnóstico de tareas generales, supera a los modelos entrenados discriminativamente que emplean arquitecturas diseñadas específicamente para cada tarea, mejorando significativamente el estado del arte en 9 de las 12 tareas estudiadas.
 * También analizamos los comportamientos de "disparo cero" del modelo preentrenado en cuatro entornos diferentes y demostramos que adquiere un conocimiento lingüístico útil para las tareas posteriores.
 * En los últimos años, los investigadores han demostrado los beneficios de utilizar incrustaciones de palabras [11, 39, 42], que se entrenan en corpus no etiquetados, para mejorar el rendimiento en una variedad de tareas [8, 11, 26, 45]. Sin embargo, estos enfoques transfieren principalmente información a nivel de palabra, mientras que nosotros pretendemos capturar la semántica de nivel superior.
 * Utilizamos un decodificador Transformer multicapa
 * 

# Fine tuning GPT

Primero hay que instalar `ftfy` y `spacy` mediante

```bash
pip install ftfy spacy
```

Una vez instaladas, debes descargar el modelo de lenguaje de spacy que deseas utilizar. Por ejemplo, para descargar el modelo de inglés, puedes ejecutar:

```bash
python -m spacy download en_core_web_sm
```

In [7]:
from transformers import OpenAIGPTConfig

config = OpenAIGPTConfig()
config

OpenAIGPTConfig {
  "afn": "gelu",
  "attn_pdrop": 0.1,
  "embd_pdrop": 0.1,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "openai-gpt",
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 12,
  "n_positions": 512,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.40.2",
  "vocab_size": 40478
}

In [37]:
from transformers import OpenAIGPTTokenizer, OpenAIGPTLMHeadModel
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")
model = OpenAIGPTLMHeadModel.from_pretrained("openai-gpt").to(device)

input_tokens = tokenizer("Hello, my dog is cute", return_tensors="pt").to(device)
output_tokens = model(**input_tokens)
output_tokens

/home/maximo.fernandez/miniconda3/envs/nlp/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


CausalLMOutput(loss=None, logits=tensor([[[ -5.9486,  -5.8697, -18.4258,  ...,  -9.7371, -10.4495,   0.8814],
         [ -6.1212,  -4.8031, -14.3970,  ...,  -6.5411,  -9.5051,  -1.2015],
         [ -7.4231,  -6.3615, -14.7297,  ..., -10.4575,  -8.4600,  -1.5183],
         [ -5.6463,  -5.9526, -17.5195,  ...,  -9.4144, -15.7120,  -1.5394],
         [ -5.4751,  -5.8803, -13.7767,  ..., -10.5048, -12.4167,  -6.1584],
         [ -7.2052,  -6.0198, -21.5040,  ..., -16.2941, -14.0494,  -1.2416]]],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>), hidden_states=None, attentions=None)

In [47]:
type(model)

transformers.models.openai.modeling_openai.OpenAIGPTLMHeadModel

In [38]:
output_tokens.keys()

odict_keys(['logits'])

In [39]:
output_tokens.logits

tensor([[[ -5.9486,  -5.8697, -18.4258,  ...,  -9.7371, -10.4495,   0.8814],
         [ -6.1212,  -4.8031, -14.3970,  ...,  -6.5411,  -9.5051,  -1.2015],
         [ -7.4231,  -6.3615, -14.7297,  ..., -10.4575,  -8.4600,  -1.5183],
         [ -5.6463,  -5.9526, -17.5195,  ...,  -9.4144, -15.7120,  -1.5394],
         [ -5.4751,  -5.8803, -13.7767,  ..., -10.5048, -12.4167,  -6.1584],
         [ -7.2052,  -6.0198, -21.5040,  ..., -16.2941, -14.0494,  -1.2416]]],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>)

In [40]:
output_tokens.logits.shape

torch.Size([1, 6, 40478])

In [41]:
input_tokens['input_ids'].shape

torch.Size([1, 6])

In [42]:
input_tokens["input_ids"][0]

tensor([3570,  240,  547, 2585,  544, 4957], device='cuda:0')

In [43]:
output_probs = torch.softmax(output_tokens.logits, dim=-1)
output_probs.shape

torch.Size([1, 6, 40478])

In [32]:
output_tokens = torch.multinomial(output_probs[0], num_samples=1)
output_tokens.shape

torch.Size([6, 1])

In [46]:
nex_token = torch.multinomial(output_probs[0, -1, :], num_samples=1)
nex_token.shape, nex_token.item()

(torch.Size([1]), 240)

In [35]:
input_token = tokenizer("Hello, my dog is cute", return_tensors="pt").to(device)
output_tokens = []

for i in range(10):  # Generar 10 tokens adicionales
    output = model(**input_token)
    output_probs = torch.softmax(output.logits, dim=-1)
    next_token = torch.multinomial(output_probs[0, -1, :], num_samples=1)
    output_tokens.append(next_token.item())
    # input_token = tokenizer.decode(next_token, return_tensors="pt").to(device)

print(output_tokens, tokenizer.decode(output_tokens))

[239, 240, 834, 240, 267, 239, 239, 240, 239, 239] ., too,!..,..
